In [1]:
import numpy as np 
import pandas as pd
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

#Library to use spotify for getting song features
import spotipy
import time
from IPython.core.display import clear_output
from spotipy import SpotifyClientCredentials, util

In [2]:
model = load_model('../Models/MMD.h5')
labels = ['Happy', 'Sad']

In [3]:
client_id='5bb8ade946c2454f8262474d0040f9c2'
client_secret='1cb00aa57c334fa2aca7a54abf987d6a'

#Credentials to access the Spotify Music Data
manager = SpotifyClientCredentials(client_id,client_secret)
sp = spotipy.Spotify(client_credentials_manager=manager)

def get_songs_features(ids):

    meta = sp.track(ids)
    features = sp.audio_features(ids)

    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']
    ids =  meta['id']

    # features
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    loudness = features[0]['loudness']
    valence = features[0]['valence']
    acousticness = features[0]['acousticness']
    instrumentalness = features[0]['instrumentalness']
    tempo = features[0]['tempo']

    track = [name, album, artist, ids, release_date, popularity, length, danceability, energy, 
             loudness, valence, acousticness, instrumentalness, tempo]
    columns = ['name','album','artist','id','release_date','popularity','length','danceability','energy',
               'loudness','valence','acousticness','instrumentalness','tempo']
    return track,columns

def get_track_id(track, artist, album):
    
    search_result = sp.search(q='artist:' + artist + ' track:' + track + ' album:' + album, type='track')
    track_id = search_result['tracks']['items'][0]['id']
    return track_id

In [4]:
def predict_mood(id_song):

    #Obtain the features of the song
    preds = get_songs_features(id_song)

    #Pre-process the features to input the Model
    preds_features = np.array(preds[0][7:]).reshape(-1,1).T

    #Predict the features of the song
    results = model.predict(preds_features)

    mood = labels[np.argmax(results)]
    name_song = preds[0][0]
    artist = preds[0][2]

    return print("{0} by {1} is a {2} song".format(name_song,artist,mood.upper()))
    #print(f"{name_song} by {artist} is a {mood[0].upper()} song")

In [5]:
track = 'Shape Of You'
artist = 'Ed Sheeran'
album = '÷'

track_id = get_track_id(track, artist, album)
predict_mood(track_id)

Shape of You by Ed Sheeran is a HAPPY song


In [6]:
track = 'Hello'
artist = 'Adele'
album = '25'

track_id = get_track_id(track, artist, album)
predict_mood(track_id)

Hello by Adele is a SAD song
